In [1]:
from pynq import Overlay
overlay = Overlay('overlay/dtp/random_forest.bit')

In [2]:
from pynq import MMIO

In [3]:
REGSBANK_BASE_ADDRESS=0x43C00000
REGSBANK_ADDRESS_RANGE=64*1024

BRAM_BASE_ADDRESS=0xC0000000
BRAM_ADDRESS_RANGE=64*1024

In [4]:
regsbank_mmio = MMIO(REGSBANK_BASE_ADDRESS, REGSBANK_ADDRESS_RANGE)

In [5]:
from enum import IntEnum
import time
from pynq import allocate
import numpy as np

In [6]:
def getbit(value, order):
    orderVal = 2**order
    tmpbin = value & orderVal
    if (tmpbin):
        return 1
    else:
        return 0

def changebit(value, order, bit):
    if (bit == 1):
        return value | (1 << order)
    else:
        return value & ~(1 << order)

class Cdma(IntEnum):
    CDMACR = 0x0
    CDMASR = 0x4
    SA = 0x18
    DA = 0x20
    BTT = 0x28

def transfer(cdma, src, dst, size):
    # Step 1
    cdmasr = cdma.read(Cdma.CDMASR)
    cdmasrIdle = getbit(cdmasr, 1)
    if (cdmasrIdle != 1):
        print("CDMA is busy..")
        return

    # Step 2
    cdmacr = cdma.read(Cdma.CDMACR)
    cdmacr = changebit(cdmacr, 12, 1) # set IOC_IrqEn
    cdmacr = changebit(cdmacr, 14, 1) # set ERR_IrqEn
    cdma.write(Cdma.CDMACR, cdmacr)

    # Step 3
    cdma.write(Cdma.SA, src)

    # Step 4
    cdma.write(Cdma.DA, dst)

    # Step 5
    cdma.write(Cdma.BTT, size)

    # Step 6
    print("Transferring...")
    cdma.read(Cdma.CDMASR)
    cdmasrIdle = getbit(cdmasr, 1)
    while (cdmasrIdle != 1):
        print(".", end="")
        cdma.read(Cdma.CDMASR)
        cdmasrIdle = getbit(cdmasr, 1)

    # Step 7-8
    cdmasr = cdma.read(Cdma.CDMASR)
    cdmasr = changebit(cdmasr, 12, 1) # clear IOC_Irq
    cdma.write(Cdma.CDMASR, cdmasr)

    print("Transfered " + str(size) + " bytes from " + str(src) + " to " + str(dst))
    print("CDMA Done.")

def reset(cdma):
    cdmacr = cdma.read(Cdma.CDMACR)
    print(type(cdmacr))
    print(cdmacr)
    cdmacr = changebit(cdmacr, 2, 1)
    print(cdmacr)
    cdma.write(Cdma.CDMACR, cdmacr)

In [11]:
for i in range(32):
    print(regsbank_mmio.read(4*i))

0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [8]:
import time

In [9]:
start_time = time.time()

regsbank_mmio.write(4*0,1) # start DTP
regsbank_mmio.write(4*0,0)

while(regsbank_mmio.read(4*2)==0):
    pass
end_time = time.time()

In [10]:
print("%s seconds for 4 samples over 25 trees" % (end_time - start_time))

0.001409292221069336 seconds for 4 samples over 25 trees


In [ ]:
accum = 0.0
for i in range(100):
    overlay = Overlay('overlay/dtp/random_forest.bit')
    regsbank_mmio = MMIO(REGSBANK_BASE_ADDRESS, REGSBANK_ADDRESS_RANGE)
    
    start_time = time.time()

    regsbank_mmio.write(4*0,1) # start DTP
    regsbank_mmio.write(4*0,0)

    while(regsbank_mmio.read(4*2)==0):
        pass
    
    end_time = time.time()
    
    accum += end_time - start_time
print("%s seconds for 4 samples over 25 trees" % (accum/100))

In [9]:
cdma = overlay.axi_cdma_0
buffer_len = 16384
output_buffer = allocate(shape=(buffer_len,), dtype=np.uint32)

In [10]:
print("==== READ CDMA ====")

# start_time = time.time()
transfer(cdma, BRAM_BASE_ADDRESS, output_buffer.physical_address, buffer_len*4)
# end_time = time.time()

==== READ CDMA ====
Transferring...
Transfered 65536 bytes from 3221225472 to 377815040
CDMA Done.


In [11]:
for i in range(400):
   print(i,output_buffer[i])

0 8
1 152
2 216
3 275
4 19
5 78
6 168
7 287
8 88
9 235
10 33
11 177
12 245
13 302
14 41
15 120
16 188
17 255
18 308
19 126
20 53
21 203
22 262
23 321
24 140
25 69
26 150
27 213
28 272
29 7
30 86
31 226
32 166
33 284
34 18
35 243
36 102
37 294
38 174
39 31
40 124
41 253
42 305
43 183
44 37
45 137
46 259
47 200
48 315
49 50
50 8
51 152
52 72
53 216
54 275
55 19
56 88
57 168
58 285
59 235
60 33
61 104
62 177
63 245
64 298
65 126
66 41
67 188
68 255
69 308
70 140
71 53
72 203
73 262
74 321
75 8
76 152
77 72
78 216
79 275
80 19
81 88
82 168
83 285
84 230
85 33
86 104
87 177
88 245
89 298
90 126
91 41
92 188
93 255
94 308
95 140
96 53
97 203
98 262
99 321
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 0
135 0
136 0
137 0
138 0
139 0
140 0
141 0
142 0
143 0
144 0
145 0
146 0
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 0
156 0
